In [7]:
# examining data files in source
import numpy as np
import pandas as pd
import pickle 
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The purpose of this notebook is to detail the process used for the ripple analysis.

For each session, we record brain activity for a certain number of trials (words). Brain activity is recorded using electrodes across various brain regions. The data for each brain region is saved separately.

Hippocampal (HPC) ripples are stored in an array of the following shape: (num_trials x num_electrodes_hpc) x num_timesteps. Here, num_trials is the number of trials, num_electrodes is the number of electrodes in that brain region, and num_timesteps is the number of timesteps that neural data was recorded for.

Let's load the HPC data from one session and see how this checks out.

In [6]:
session_events_hpc = pd.DataFrame()
ripple_array_hpc = []
sub_sess_names_hpc = []
sub_names_hpc = []
trial_nums_hpc = []
elec_ripple_rate_array_hpc = []

fn_hpc = '/scratch/john/SWR_scratch/ENCODING/SWR_catFR1_R1015J_0_HPC_encoding_soz_in_hamming.p'
with open(fn_hpc,'rb') as f:
    dat = pickle.load(f)
    session_events_hpc = session_events_hpc.append(dat['session_events'])
    ripple_array_hpc = superVstack(ripple_array_hpc,dat['ripple_array'])
    sub_sess_names_hpc.extend(dat['sub_sess_names'])
    sub_names_hpc.extend(dat['sub_names'])
    trial_nums_hpc = np.append(trial_nums_hpc,dat['trial_nums'])
    elec_ripple_rate_array_hpc.extend(dat['elec_ripple_rate_array'])
    regions_hpc = dat['HPC_names']
    
print(f"Ripple array shape: {ripple_array_hpc.shape}")
print(f"Electrode regions: {regions_hpc}")
print(f"Trials recorded from each electrode: {trial_nums_hpc}")

Ripple array shape: (840, 1500)
Electrode regions: ['left sub', 'left ca1', 'left ca1', 'left sub', 'left ca1', 'left dg', 'left ca1']
Trials recorded from each electrode: [120. 120. 120. 120. 120. 120. 120.]


We can see that for this given session, the participant encoded 120 words. While doing so, brain activity was recorded from 7 different subregions in HPC. That equates to a total of 7*120, or 840 trials of data.



Suppose we're interested in analyzing amygdala (AMY) high-frequency activity data when a ripple happens in the CA1 region of HPC. Let's load the AMY data first to get started. 

In [9]:
session_events_amy = pd.DataFrame()
ripple_array_amy = []
sub_sess_names_amy = []
sub_names_amy = []
trial_nums_amy = []
elec_ripple_rate_array_amy = []

fn_amy = '/scratch/john/SWR_scratch/ENCODING/SWR_catFR1_R1015J_0_AMY_encoding_soz_in_hamming.p'
with open(fn_amy,'rb') as f:
    dat = pickle.load(f)
    session_events_amy = session_events_amy.append(dat['session_events'])
    ripple_array_amy = superVstack(ripple_array_amy,dat['ripple_array'])
    sub_sess_names_amy.extend(dat['sub_sess_names'])
    sub_names_amy.extend(dat['sub_names'])
    trial_nums_amy = np.append(trial_nums_amy,dat['trial_nums'])
    elec_ripple_rate_array_amy.extend(dat['elec_ripple_rate_array']) 
    regions_amy = dat['HPC_names']
    
print(f"Ripple array shape: {ripple_array_amy.shape}")
print(f"Electrode regions: {regions_amy}")
print(f"Trials recorded from each electrode: {trial_nums_amy}")

Ripple array shape: (240, 1500)
Electrode regions: ['left amy', 'left amy']
Trials recorded from each electrode: [120. 120.]


We can see that AMY also has a ripple array, but we're not interested in this because we want to condition on CA1 ripples from the same hemisphere. Let's take only CA1 ripples in the next cell, and separate them based on the hemisphere they are in.

In [14]:
hpc_subregions_selected = ['ca1'] # only want ripples from this region of hpc 

# loop through hpc regions
# store ripples from the selected
left_hemi_data = []
right_hemi_data = []
trialStart = 0
for region, trials in zip(regions_hpc, trial_nums_hpc):
    trials = int(trials)
    for sb in hpc_subregions_selected:
        if f'left {sb}' in region: 
            left_hemi_data.append(ripple_array_hpc[trialStart:trialStart+trials])
        if f'right {sb}' in region: 
            right_hemi_data.append(ripple_array_hpc[trialStart:trialStart+trials])
        
        
print(f"CA1 ripple array left hemisphere shape: {np.stack(left_hemi_data).shape}")

CA1 ripple array left hemisphere shape: (4, 120, 1500)


We can see that, for the left hemisphere, we have an array of shape num_electrodes x num_trials x num_timesteps. Here, num_electrodes is the number of electrodes from selected subregions from that hemisphere. 

In [17]:
def ripples_hpc(nonhpc_trialnums, nonhpc_regions, hpc_trialnums, hpc_regions, ripple_array_hpc, regions_selected):

    
    timesteps = ripple_array_hpc.shape[1]
    num_trials = np.sum(nonhpc_trialnums)

    # Step 1: Separate hpc right and left ripples
    left_hemi_data = []
    right_hemi_data = []
    trialStart = 0
    for region, trials in zip(hpc_regions, hpc_trialnums):
        trials = int(trials)
        for rs in regions_selected:
            if f'right {rs}' in region: 
                right_hemi_data.append(ripple_array_hpc[trialStart:trialStart+trials])
            if f'left {rs}' in region: 
                left_hemi_data.append(ripple_array_hpc[trialStart:trialStart+trials])
        trialStart += trials 

    if len(right_hemi_data) > 0:
        rhd_np = np.clip(np.sum(np.stack(right_hemi_data),axis=0), a_min=0, a_max=1)
    else:
        rhd_np = np.zeros_like((trials, timesteps))
    if len(left_hemi_data) > 0:
        lhd_np = np.clip(np.sum(np.stack(left_hemi_data),axis=0), a_min=0, a_max=1)
    else:
        lhd_np = np.zeros_like((trials, timesteps))


    ripples_hpc_nonhpcshape = np.zeros((int(num_trials), timesteps))

    trialStart = 0
    for region, trials in zip(nonhpc_regions, nonhpc_trialnums):
        trials = int(trials)
        
        if 'right' in region:
            ripples_hpc_nonhpcshape[trialStart:trialStart+trials] = rhd_np
        if 'left' in region:
            ripples_hpc_nonhpcshape[trialStart:trialStart+trials] = lhd_np

        trialStart += trials 

    return ripples_hpc_nonhpcshape



In [24]:
ripples_hpc_amyshape = ripples_hpc(trial_nums_amy, regions_amy,
                                 trial_nums_hpc, regions_hpc, ripple_array_hpc, ['ca1'])

In [27]:
np.where(np.sum(ripples_hpc_amyshape, axis=1)==0, x>=1


(240,)